# Business Problem section


### Background

According to Bloomberg News, the London Housing Market is in a rut. It is now facing a number of different headwinds, including the prospect of higher taxes and a warning from the Bank of England that U.K. home values could fall as much as 30 percent in the event of a disorderly exit from the European Union. More specifically, four overlooked cracks suggest that the London market may be in worse shape than many realize: hidden price falls, record-low sales, homebuilder exodus and tax hikes addressing overseas buyers of homes in England and Wales.  

As of March 2020 the average house price in the UK is £231,855, and the index stands at 121.6. Property prices have fallen by 0.2% compared to the previous month, and risen by 2.1% compared to the previous year.   

It is advised that low number of sales transactions in some local authorities can lead to volatility in the estimates at these levels. Geographies with low number of sales transactions should be analysed in the context of their longer-term trends rather than focusing on monthly movements. Our guidance page provides further detail about our data and its quality.  

Historic data within this tool is derived. Under the UK HPI, data is available from 1995 for England and Wales, 2004 for Scotland and 2005 for Northern Ireland. A longer back-series has been derived by using the historic path of the Office for National Statistics HPI to construct a series back to 1968.


### Business Problem
In this scenario, it is urgent to adopt machine learning tools in order to assist **homebuyers clientele in London** to make wise and effective decisions. As a result, the business problem we are currently posing is: how could we provide support to homebuyers clientele in to purchase a suitable real estate in London in this uncertain economic and financial scenario?

To solve this business problem, we are going to cluster London neighborhoods in order to recommend venues and the current average price of real estate where homebuyers can make a real estate investment. We will recommend profitable venues according to amenities and essential facilities surrounding such venues i.e. elementary schools, high schools, hospitals & grocery stores.

## Data section
Data on London properties and the relative price paid data were extracted from the HM Land Registry (http://landregistry.data.gov.uk/). The following fields comprise the address data included in **Price Paid Data**:(targe value) **Postcode;** PAON Primary Addressable Object Name. Typically the house number or name; SAON Secondary Addressable Object Name. If there is a sub-building, for example, the building is divided into **flats**, there will be a SAON; **Street**; **Locality;** **Town/City;** **District;** **County**.

To explore and target recommended locations across different venues according to the presence of amenities and essential facilities, we will access data through FourSquare API interface and arrange them as a dataframe for visualization. By merging data on London properties and the relative price paid data from the HM Land Registry and data on amenities and essential facilities surrounding such properties from FourSquare API interface, we will be able to recommend profitable real estate investments.



 # Methodology section  
 
The Methodology section will describe the main components of our analysis and predication system. The Methodology section comprises four stages:

1. Collect Inspection Data
2. Explore and Understand Data
3. Data preparation and preprocessing 
4. Modeling

1. Collect Inspection Data  
After importing the necessary libraries, we download the data from the HM Land Registry website as follows:

In [1]:
import os # Operating System
import numpy as np
import pandas as pd
import datetime as dt # Datetime
import json # library to handle JSON files

!pip install geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

!pip install  folium 
import folium #import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [2]:
#Read the data for examination (Source: http://landregistry.data.gov.uk/)
df_prod = pd.read_csv("http://prod2.publicdata.landregistry.gov.uk.s3-website-eu-west-1.amazonaws.com/pp-2018.csv")
#df_prod = pd.read_csv("pp2018.csv")

In [3]:
#df_prod.to_csv('pp2018.csv')

Before using data, we will have to explore and understand it.

2. Explore and Understand Data  
We read the dataset that we collected from the HM Land Registry website into a pandas' data frame and display the first five rows of it as follows:

In [4]:
#df_prod= df_prod.drop(["Unnamed: 0"],axis= 1)
df_prod.head()

,{79A74E21-D11E-1289-E053-6B04A8C01627},770000,2018-09-25 00:00,SK7 1AR,D,N,F,5,Unnamed: 8,OAK MEADOW,BRAMHALL,STOCKPORT,STOCKPORT.1,GREATER MANCHESTER,A,A.1
0,{79A74E21-D11F-1289-E053-6B04A8C01627},253500,2018-09-24 00:00,M6 8GQ,D,N,F,1,NaN,RIVINGTON ROAD,NaN,SALFORD,SALFORD,GREATER MANCHESTER,A,A
1,{79A74E21-D120-1289-E053-6B04A8C01627},231950,2018-09-28 00:00,WA3 2UE,D,Y,F,35,NaN,STONEACRE CLOSE,LOWTON,WARRINGTON,WIGAN,GREATER MANCHESTER,A,A
2,{79A74E21-D121-1289-E053-6B04A8C01627},112500,2018-08-29 00:00,OL6 6RJ,S,N,F,102,NaN,THORNFIELD GROVE,NaN,ASHTON-UNDER-LYNE,TAMESIDE,GREATER MANCHESTER,A,A
3,{79A74E21-D122-1289-E053-6B04A8C01627},184995,2018-06-15 00:00,M46 0TW,S,Y,F,37,NaN,THREADNEEDLE PLACE,ATHERTON,MANCHESTER,WIGAN,GREATER MANCHESTER,A,A
4,{79A74E21-D123-1289-E053-6B04A8C01627},214995,2018-09-28 00:00,M28 3XS,D,Y,L,9,NaN,MARPLE GARDENS,WORSLEY,MANCHESTER,SALFORD,GREATER MANCHESTER,A,A


In [5]:
df_prod.shape

(1029929, 16)

Our dataset consists of over 700000 rows and 16 columns. We will now prepare and preprocess data accordingly.

In [6]:
df_prod.describe()

,770000
count,1.029929e+06
mean,3.494193e+05
std,1.843504e+06
min,1.000000e+00
25%,1.450000e+05
50%,2.300000e+05
75%,3.550000e+05
max,5.692000e+08



3. Data preparation and preprocessing
At this stage, we prepare our dataset for the modeling process, opting for the most suitable machine learning algorithm for our scope. Accordingly, we perform the following steps:
            
  <ul>
        <li>Rename the column names  </li>
        <li>Format the date column  </li>
        <li>Sort data by date of sale  </li>
        <li>Select data only for the city of London  </li>
        <li>Make a list of street names in London  </li>
        <li>Calculate the street-wise average price of the property  </li>
        <li>Read the street-wise coordinates into a data frame, eliminating recurring word London from individual names  </li>
        <li>Join the data to find the coordinates of locations which fit into client's budget  </li>
        <li>Plot recommended locations on London map along with current market prices </li>
       </ul>

In [7]:
# Assign meaningful column names
df_prod.columns = ['TUID', 'Price', 'Date_Transfer', 'Postcode', 'Prop_Type', 'Old_New', 'Duration', 'PAON', \
                  'SAON', 'Street', 'Locality', 'Town_City', 'District', 'County', 'PPD_Cat_Type', 'Record_Status']
df_prod.head()

,TUID,Price,Date_Transfer,Postcode,Prop_Type,Old_New,Duration,PAON,SAON,Street,Locality,Town_City,District,County,PPD_Cat_Type,Record_Status
0,{79A74E21-D11F-1289-E053-6B04A8C01627},253500,2018-09-24 00:00,M6 8GQ,D,N,F,1,NaN,RIVINGTON ROAD,NaN,SALFORD,SALFORD,GREATER MANCHESTER,A,A
1,{79A74E21-D120-1289-E053-6B04A8C01627},231950,2018-09-28 00:00,WA3 2UE,D,Y,F,35,NaN,STONEACRE CLOSE,LOWTON,WARRINGTON,WIGAN,GREATER MANCHESTER,A,A
2,{79A74E21-D121-1289-E053-6B04A8C01627},112500,2018-08-29 00:00,OL6 6RJ,S,N,F,102,NaN,THORNFIELD GROVE,NaN,ASHTON-UNDER-LYNE,TAMESIDE,GREATER MANCHESTER,A,A
3,{79A74E21-D122-1289-E053-6B04A8C01627},184995,2018-06-15 00:00,M46 0TW,S,Y,F,37,NaN,THREADNEEDLE PLACE,ATHERTON,MANCHESTER,WIGAN,GREATER MANCHESTER,A,A
4,{79A74E21-D123-1289-E053-6B04A8C01627},214995,2018-09-28 00:00,M28 3XS,D,Y,L,9,NaN,MARPLE GARDENS,WORSLEY,MANCHESTER,SALFORD,GREATER MANCHESTER,A,A


In [8]:

# Format the date column
df_prod['Date_Transfer'] = df_prod['Date_Transfer'].apply(pd.to_datetime)

# Delete all obsolete transactions which were done before 2016
df_prod.drop(df_prod[df_prod.Date_Transfer.dt.year < 2016].index, inplace=True)

# Sort by Date of Sale
df_prod.sort_values(by=['Date_Transfer'],ascending=[False],inplace=True)
df_prod.head()

,TUID,Price,Date_Transfer,Postcode,Prop_Type,Old_New,Duration,PAON,SAON,Street,Locality,Town_City,District,County,PPD_Cat_Type,Record_Status
936864,{80E1AA99-04FF-7BF8-E053-6C04A8C00BF2},8050822,2018-12-31,HP2 5NP,O,N,F,"QUEENSWAY HOUSE, 148",NaN,JUPITER DRIVE,NaN,HEMEL HEMPSTEAD,DACORUM,HERTFORDSHIRE,B,A
807430,{85866A64-5FF9-143F-E053-6B04A8C06A15},540000,2018-12-31,UB10 0FN,T,Y,F,67,NaN,CHURCHILL ROAD,NaN,UXBRIDGE,HILLINGDON,GREATER LONDON,A,A
684367,{9DBAD222-965D-6EB3-E053-6B04A8C0F257},250000,2018-12-31,S41 9QB,O,N,F,UNIVERSAL HYDRAULICS LTD,NaN,CARRWOOD ROAD,NaN,CHESTERFIELD,CHESTERFIELD,DERBYSHIRE,B,A
300725,{87E1551E-887E-6405-E053-6C04A8C0B2EE},399995,2018-12-31,GU34 2FX,D,Y,F,7,NaN,MORAY PLACE,NaN,ALTON,EAST HAMPSHIRE,HAMPSHIRE,A,A
154720,{8355F008-D7B6-55C5-E053-6B04A8C0D090},339000,2018-12-31,PL9 8FH,T,Y,F,8,NaN,AQUARIUS DRIVE,SHERFORD,PLYMOUTH,SOUTH HAMS,DEVON,A,A


In [9]:
#sellecting all city with london name 
df_london = df_prod.query("Town_City == 'LONDON'")

# Make a list of street names in LONDON
streets = df_london['Street'].unique().tolist()
streets

['BRIDGE LANE',
 'HERMITAGE ROAD',
 'CAMLEY STREET',
 'ST JOSEPHS STREET',
 'SHEPHERDESS WALK',
 'DERWENT ROAD',
 'VICTORIA STREET',
 'BROSTER GARDENS',
 'SYLVAN HILL',
 'THREE OAK LANE',
 'GLASSHOUSE GARDENS',
 'QUICKS ROAD',
 'ST GEORGES SQUARE',
 'HYDE ESTATE ROAD',
 'RITHERDON ROAD',
 'TRINITY CRESCENT',
 'APPLE YARD',
 'LANCASTER GATE',
 'HANDYSIDE STREET',
 'MICHLEHAM DOWN',
 'CHINGFORD MOUNT ROAD',
 'GRAND DRIVE',
 'EXCHANGE GARDENS',
 'BRUSHFIELD STREET',
 'WENTWORTH STREET',
 'BUCKINGHAM PALACE ROAD',
 'CLIFFORD STREET',
 'HOWARD ROAD',
 'BLACKFRIARS ROAD',
 'HIGHWOOD HILL',
 "ST JAMES'S PLACE",
 'FLORAL STREET',
 'RYMILL STREET',
 'COLLENDALE ROAD',
 'TURNHAM GREEN TERRACE',
 'PALACE COURT',
 'HONOUR LEA AVENUE',
 'ARCHWAY ROAD',
 'HARTFIELD ROAD',
 'GALLOWAY ROAD',
 'WARWICK WAY',
 'OXFORD STREET',
 'UPPER TERRACE',
 'WOOLF CLOSE',
 'FULHAM HIGH STREET',
 'ABBOTS MANOR',
 'LOWER BOSTON ROAD',
 'PRINCE ALBERT ROAD',
 'UPPER RICHMOND ROAD',
 'NORTH BIRKBECK ROAD',
 'CLIFFORD R

In [10]:
#group the london data with the home street and calculate the average for each one  
df_price = df_london.groupby(['Street'])['Price'].mean().reset_index()

# Give meaningful names to the columns
df_price.columns = ['Street', 'Avg_Price']

In [11]:
#Input your Budget's Upper Limit and Lower Limit - Find the locations df_grp_price which fits your budget
df_affordable = df_price.query("(Avg_Price >= 2200000) & (Avg_Price <= 2500000)")

# Display the dataframe
df_affordable

,Street,Avg_Price
196,ALBION SQUARE,2.450000e+06
390,ANHALT ROAD,2.435000e+06
405,ANSDELL TERRACE,2.250000e+06
422,APPLEGARTH ROAD,2.400000e+06
855,BARONSMEAD ROAD,2.375000e+06
981,BEAUCLERC ROAD,2.480000e+06
1102,BELVEDERE DRIVE,2.340000e+06
1215,BICKENHALL STREET,2.208500e+06
1253,BIRCHLANDS AVENUE,2.217000e+06
1553,BRAMPTON GROVE,2.456875e+06


In [13]:

import pandas as pd
import numpy as np
import datetime as DT
import hmac
from geopy.geocoders import Nominatim
from geopy.distance import great_circle
# import k-means from clustering stage
from sklearn.cluster import KMeans

In [14]:
for index, item in df_affordable.iterrows():
    print(f"index: {index}")
    print(f"item: {item}")
    print(f"item.Street only: {item.Street}")

index: 196
item: Street       ALBION SQUARE
Avg_Price         2.45e+06
Name: 196, dtype: object
item.Street only: ALBION SQUARE
index: 390
item: Street       ANHALT ROAD
Avg_Price      2.435e+06
Name: 390, dtype: object
item.Street only: ANHALT ROAD
index: 405
item: Street       ANSDELL TERRACE
Avg_Price           2.25e+06
Name: 405, dtype: object
item.Street only: ANSDELL TERRACE
index: 422
item: Street       APPLEGARTH ROAD
Avg_Price            2.4e+06
Name: 422, dtype: object
item.Street only: APPLEGARTH ROAD
index: 855
item: Street       BARONSMEAD ROAD
Avg_Price          2.375e+06
Name: 855, dtype: object
item.Street only: BARONSMEAD ROAD
index: 981
item: Street       BEAUCLERC ROAD
Avg_Price          2.48e+06
Name: 981, dtype: object
item.Street only: BEAUCLERC ROAD
index: 1102
item: Street       BELVEDERE DRIVE
Avg_Price           2.34e+06
Name: 1102, dtype: object
item.Street only: BELVEDERE DRIVE
index: 1215
item: Street       BICKENHALL STREET
Avg_Price           2.2085e+06
N

In [15]:
geolocator = Nominatim()
df_affordable['city_coord'] = df_affordable['Street'].apply(geolocator.geocode).apply(lambda x: (x.latitude, x.longitude))


/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:1: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  if __name__ == '__main__':
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [16]:
df_affordable


,Street,Avg_Price,city_coord
196,ALBION SQUARE,2.450000e+06,"(-41.27375755, 173.28939323910353)"
390,ANHALT ROAD,2.435000e+06,"(51.4803164, -0.1668011)"
405,ANSDELL TERRACE,2.250000e+06,"(51.4998899, -0.1891027)"
422,APPLEGARTH ROAD,2.400000e+06,"(53.7486539, -0.3266704)"
855,BARONSMEAD ROAD,2.375000e+06,"(51.4773147, -0.239457)"
981,BEAUCLERC ROAD,2.480000e+06,"(30.2114523, -81.6179807)"
1102,BELVEDERE DRIVE,2.340000e+06,"(44.7075622, -63.5455988)"
1215,BICKENHALL STREET,2.208500e+06,"(51.5212014, -0.1589082)"
1253,BIRCHLANDS AVENUE,2.217000e+06,"(51.4483941, -0.1604676)"
1553,BRAMPTON GROVE,2.456875e+06,"(51.5899607, -0.3185249)"


In [17]:
df_affordable[['Latitude', 'Longitude']] =  df_affordable['city_coord'].apply(pd.Series)

/opt/conda/envs/Python36/lib/python3.6/site-packages/pandas/core/frame.py:3391: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


In [18]:

df = df_affordable.drop(['city_coord'] , axis= 1 )
df_affordable.head()

,Street,Avg_Price,city_coord,Latitude,Longitude
196,ALBION SQUARE,2450000.0,"(-41.27375755, 173.28939323910353)",-41.273758,173.289393
390,ANHALT ROAD,2435000.0,"(51.4803164, -0.1668011)",51.480316,-0.166801
405,ANSDELL TERRACE,2250000.0,"(51.4998899, -0.1891027)",51.499890,-0.189103
422,APPLEGARTH ROAD,2400000.0,"(53.7486539, -0.3266704)",53.748654,-0.326670
855,BARONSMEAD ROAD,2375000.0,"(51.4773147, -0.239457)",51.477315,-0.239457


In [19]:
df.head()

,Street,Avg_Price,Latitude,Longitude
196,ALBION SQUARE,2450000.0,-41.273758,173.289393
390,ANHALT ROAD,2435000.0,51.480316,-0.166801
405,ANSDELL TERRACE,2250000.0,51.499890,-0.189103
422,APPLEGARTH ROAD,2400000.0,53.748654,-0.326670
855,BARONSMEAD ROAD,2375000.0,51.477315,-0.239457


In [20]:
address = 'London, UK'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of London City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of London City are 51.5073219, -0.1276474.


/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  app.launch_new_instance()


In [21]:
# create map of London using latitude and longitude values
map_london = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, price, street in zip(df['Latitude'], df['Longitude'], df['Avg_Price'], df['Street']):
    label = '{}, {}'.format(street, price)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_london)  
    
map_london

In [22]:
#Define Foursquare Credentials and Version

CLIENT_ID = '0SBKYDIOD1KDVN4JSSAXT55VFU1U3WE22SKNES1XJNVGIFN4' # Foursquare ID
CLIENT_SECRET = 'TL1JTJD23XIVZJNIM2JSYWHYMJRP5BDH5EQV1IFBLHKACER3' # Foursquare Secret
VERSION = '20200630' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 0SBKYDIOD1KDVN4JSSAXT55VFU1U3WE22SKNES1XJNVGIFN4
CLIENT_SECRET:TL1JTJD23XIVZJNIM2JSYWHYMJRP5BDH5EQV1IFBLHKACER3



We can now proceed to the Modeling phase. We will analyze neighborhoods to recommend real estates where home buyers can make a real estate investment. We will then recommend profitable venues according to amenities and essential facilities surrounding such venues i.e. elementary schools, high schools, hospitals & grocery stores.  

## 4. Modeling

After exploring the dataset and gaining insights into it, we are ready to use the clustering methodology to analyze real estates. We will use the k-means clustering technique as it is fast and efficient in terms of computational cost, is highly flexible to account for mutations in real estate market in London and is accurate.


In [23]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Street', 
                  'Street Latitude', 
                  'Street Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [24]:
# Run the above function on each location and create a new dataframe called location_venues and display it.
location_venues = getNearbyVenues(names=df['Street'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

ALBION SQUARE
ANHALT ROAD
ANSDELL TERRACE
APPLEGARTH ROAD
BARONSMEAD ROAD
BEAUCLERC ROAD
BELVEDERE DRIVE
BICKENHALL STREET
BIRCHLANDS AVENUE
BRAMPTON GROVE
BRIARDALE GARDENS
BROOKWAY
BURBAGE ROAD
BURY WALK
CALLCOTT STREET
CAMPDEN HILL ROAD
CAMPION ROAD
CANNING PLACE
CARLISLE ROAD
CARLTON GARDENS
CARLYLE COURT
CHALCOT SQUARE
CHARLES LANE
CHELSEA CRESCENT
CHESTER CLOSE NORTH
CHEYNE COURT
CHEYNE ROW
CHISWICK MALL
CITY ROAD
CLARENDON STREET
CLONCURRY STREET
COLBECK MEWS
COLLEGE CRESCENT
CORNWALL TERRACE MEWS
COURT LANE GARDENS
CRESCENT GROVE
DALEBURY ROAD
DEWHURST ROAD
DORIA ROAD
DOWNSHIRE HILL
DUCHESS WALK
ECCLESTON SQUARE MEWS
EGBERT STREET
EGERTON PLACE
ELM PARK ROAD
FLORAL STREET
FRANK DIXON WAY
FULTON MEWS
GERARD ROAD
GERRARD ROAD
GIRDLERS ROAD
GLOUCESTER CRESCENT
GORDON PLACE
GRAFTON SQUARE
GRAHAM TERRACE
HARMAN DRIVE
HARRIS STREET
HAVANNAH STREET
HAZLEWELL ROAD
HEREFORD MEWS
HERONDALE AVENUE
HIGHGATE HIGH STREET
HIGHWOOD HILL
HILLGATE PLACE
HOLLYCROFT AVENUE
HOLLYWOOD MEWS
HONEYWELL

In [25]:
location_venues.head()

,Street,Street Latitude,Street Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,ALBION SQUARE,-41.273758,173.289393,The Free House,-41.273340,173.287364,Bar
1,ALBION SQUARE,-41.273758,173.289393,Queen's Gardens,-41.273671,173.291383,Park
2,ALBION SQUARE,-41.273758,173.289393,The Indian Cafe,-41.273308,173.286530,Indian Restaurant
3,ALBION SQUARE,-41.273758,173.289393,Urban,-41.274355,173.286317,New American Restaurant
4,ALBION SQUARE,-41.273758,173.289393,Fish Stop,-41.276010,173.289592,Fish & Chips Shop


In [26]:
location_venues.groupby('Street').count()

,Street Latitude,Street Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Street,,,,,,
ALBION SQUARE,27,27,27,27,27,27
ANHALT ROAD,16,16,16,16,16,16
ANSDELL TERRACE,43,43,43,43,43,43
APPLEGARTH ROAD,6,6,6,6,6,6
BARONSMEAD ROAD,14,14,14,14,14,14
BEAUCLERC ROAD,4,4,4,4,4,4
BELVEDERE DRIVE,1,1,1,1,1,1
BICKENHALL STREET,62,62,62,62,62,62
BIRCHLANDS AVENUE,9,9,9,9,9,9


In [27]:
# get the List of Unique Categories
print('There are {} uniques categories.'.format(len(location_venues['Venue Category'].unique())))
location_venues.shape

There are 347 uniques categories.


(4494, 7)

In [28]:


# one hot encoding
venues_onehot = pd.get_dummies(location_venues[['Venue Category']], prefix="", prefix_sep="")

# add street column back to dataframe
venues_onehot['Street'] = location_venues['Street'] 

# move street column to the first column
fixed_columns = [venues_onehot.columns[-1]] + list(venues_onehot.columns[:-1])

#fixed_columns
venues_onehot = venues_onehot[fixed_columns]

venues_onehot.head()



,Street,ATM,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arcade,Argentinian Restaurant,...,Vietnamese Restaurant,Warehouse Store,Waterfront,Weight Loss Center,Wine Bar,Wine Shop,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
0,ALBION SQUARE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ALBION SQUARE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ALBION SQUARE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ALBION SQUARE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ALBION SQUARE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [29]:
london_grouped = venues_onehot.groupby('Street').mean().reset_index()
london_grouped

,Street,ATM,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arcade,Argentinian Restaurant,...,Vietnamese Restaurant,Warehouse Store,Waterfront,Weight Loss Center,Wine Bar,Wine Shop,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
0,ALBION SQUARE,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,...,0.000000,0.00,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0
1,ANHALT ROAD,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,...,0.000000,0.00,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0
2,ANSDELL TERRACE,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,...,0.000000,0.00,0.000000,0.0,0.023256,0.0,0.000000,0.000000,0.0,0.0
3,APPLEGARTH ROAD,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,...,0.000000,0.00,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0
4,BARONSMEAD ROAD,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,...,0.000000,0.00,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0
5,BEAUCLERC ROAD,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,...,0.000000,0.00,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0
6,BELVEDERE DRIVE,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,...,0.000000,0.00,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0
7,BICKENHALL STREET,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,...,0.016129,0.00,0.000000,0.0,0.016129,0.0,0.016129,0.016129,0.0,0.0
8,BIRCHLANDS AVENUE,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,...,0.000000,0.00,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0
9,BRIARDALE GARDENS,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,...,0.000000,0.00,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0


In [30]:
london_grouped.shape

(149, 348)

In [31]:
# What are the top 5 venues/facilities nearby profitable real estate investments?#

num_top_venues = 5

for hood in london_grouped['Street']:
    print("----"+hood+"----")
    temp = london_grouped[london_grouped['Street'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----ALBION SQUARE----
               venue  freq
0               Café  0.19
1         Restaurant  0.07
2                Bar  0.07
3                Pub  0.07
4  Indian Restaurant  0.07


----ANHALT ROAD----
                  venue  freq
0                   Pub  0.25
1     French Restaurant  0.12
2         Grocery Store  0.12
3   Japanese Restaurant  0.06
4  Gym / Fitness Center  0.06


----ANSDELL TERRACE----
               venue  freq
0          Juice Bar  0.09
1                Pub  0.07
2         Restaurant  0.07
3              Hotel  0.07
4  Indian Restaurant  0.05


----APPLEGARTH ROAD----
            venue  freq
0             Bar  0.33
1       Nightclub  0.17
2          Casino  0.17
3             Pub  0.17
4  Sandwich Place  0.17


----BARONSMEAD ROAD----
               venue  freq
0  Food & Drink Shop  0.14
1         Restaurant  0.07
2     Breakfast Spot  0.07
3        Coffee Shop  0.07
4      Movie Theater  0.07


----BEAUCLERC ROAD----
             venue  freq
0              Spa

                 venue  freq
0           Donut Shop  0.09
1        Jewelry Store  0.09
2          Coffee Shop  0.09
3                  Pub  0.04
4  Szechuan Restaurant  0.04


----FRANK DIXON WAY----
                  venue  freq
0                  Café   0.2
1  Gym / Fitness Center   0.2
2                  Farm   0.2
3                  Park   0.2
4           Rugby Pitch   0.2


----FULTON MEWS----
                venue  freq
0               Hotel  0.15
1                 Pub  0.08
2         Coffee Shop  0.07
3                Café  0.05
4  Chinese Restaurant  0.05


----GERARD ROAD----
                        venue  freq
0           Convenience Store  0.25
1  Construction & Landscaping  0.25
2               Grocery Store  0.25
3           Fish & Chips Shop  0.25
4                         ATM  0.00


----GERRARD ROAD----
                        venue  freq
0  Construction & Landscaping  0.33
1                         Pub  0.33
2        Fast Food Restaurant  0.33
3                        

                        venue  freq
0  Construction & Landscaping  0.25
1           Indian Restaurant  0.25
2               Grocery Store  0.25
3            Business Service  0.25
4                         ATM  0.00


----MUNDEN STREET----
               venue  freq
0              Hotel  0.09
1               Café  0.09
2                Pub  0.09
3  Indian Restaurant  0.08
4        Pizza Place  0.06


----NORTH CIRCULAR ROAD----
                         venue  freq
0               Shopping Plaza   0.1
1                  Pizza Place   0.1
2       Furniture / Home Store   0.1
3            Electronics Store   0.1
4  Eastern European Restaurant   0.1


----NOTTINGHAM STREET----
                  venue  freq
0     Convenience Store  0.33
1      Halal Restaurant  0.17
2     Indian Restaurant  0.17
3          Home Service  0.17
4  Gym / Fitness Center  0.17


----OAKWOOD COURT----
         venue  freq
0  Video Store   0.2
1         Food   0.2
2   Smoke Shop   0.2
3  Golf Course   0.2
4  Pizza 

                venue  freq
0  Italian Restaurant  0.23
1      Ice Cream Shop  0.11
2               Hotel  0.09
3                Café  0.08
4      Sandwich Place  0.04


----STAFFORD TERRACE----
                  venue  freq
0           Supermarket  0.38
1   Rental Car Location  0.12
2             Pet Store  0.12
3                   Pub  0.12
4  Fast Food Restaurant  0.12


----SUTHERLAND PLACE----
              venue  freq
0              Café  0.17
1      Burger Joint  0.17
2              Park  0.17
3  Sushi Restaurant  0.17
4         Gastropub  0.17


----SYDNEY STREET----
                     venue  freq
0                      Gym   0.5
1              Men's Store   0.5
2                      ATM   0.0
3  New American Restaurant   0.0
4             Perfume Shop   0.0


----THAMES BANK----
                  venue  freq
0  Gym / Fitness Center  0.25
1          Burger Joint  0.25
2         Grocery Store  0.25
3           Pizza Place  0.25
4                   ATM  0.00


----THE HEXAGON-

In [32]:


# Define a function to return the most common venues/facilities nearby real estate investments#

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]



In [33]:


num_top_venues = 7

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Street']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))



In [34]:
# create a new dataframe
venues_sorted = pd.DataFrame(columns=columns)
venues_sorted['Street'] = london_grouped['Street']

for ind in np.arange(london_grouped.shape[0]):
    venues_sorted.iloc[ind, 1:] = return_most_common_venues(london_grouped.iloc[ind, :], num_top_venues)

venues_sorted.head()

,Street,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
0,ALBION SQUARE,Café,Coffee Shop,Indian Restaurant,Pub,Restaurant,Bar,Park
1,ANHALT ROAD,Pub,French Restaurant,Grocery Store,English Restaurant,Diner,Cocktail Bar,Plaza
2,ANSDELL TERRACE,Juice Bar,Hotel,Pub,Restaurant,Indian Restaurant,Italian Restaurant,Clothing Store
3,APPLEGARTH ROAD,Bar,Pub,Casino,Nightclub,Sandwich Place,Zoo Exhibit,Farmers Market
4,BARONSMEAD ROAD,Food & Drink Shop,Nature Preserve,Restaurant,Farmers Market,Coffee Shop,Park,Pizza Place


In [35]:
venues_sorted.shape

(149, 8)

In [36]:
london_grouped.shape

(149, 348)

In [37]:
london_grouped=df


**After our inspection of venues/facilities/amenities nearby the most profitable real estate investments in London, we could begin by clustering properties by venues/facilities/amenities nearby.**

In [38]:
#Distribute in 5 Clusters

# set number of clusters
kclusters = 5

london_grouped_clustering = london_grouped.drop('Street', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(london_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:50]

array([1, 3, 0, 3, 2, 1, 2, 0, 0, 1, 3, 3, 3, 1, 2, 2, 1, 3, 0, 1, 4, 4,
       3, 1, 1, 0, 3, 4, 1, 0, 3, 2, 3, 2, 2, 4, 3, 3, 2, 0, 1, 2, 4, 0,
       4, 0, 0, 4, 0, 0], dtype=int32)

In [39]:
#Dataframe to include Clusters

london_grouped_clustering=df
london_grouped_clustering.head()

,Street,Avg_Price,Latitude,Longitude
196,ALBION SQUARE,2450000.0,-41.273758,173.289393
390,ANHALT ROAD,2435000.0,51.480316,-0.166801
405,ANSDELL TERRACE,2250000.0,51.499890,-0.189103
422,APPLEGARTH ROAD,2400000.0,53.748654,-0.326670
855,BARONSMEAD ROAD,2375000.0,51.477315,-0.239457


In [40]:
london_grouped_clustering.shape

(162, 4)

In [41]:
london_grouped_clustering.dtypes

Street        object
Avg_Price    float64
Latitude     float64
Longitude    float64
dtype: object

In [42]:
df.shape


(162, 4)

In [43]:
df.dtypes

Street        object
Avg_Price    float64
Latitude     float64
Longitude    float64
dtype: object

In [44]:
# add clustering labels
london_grouped_clustering['Cluster Labels'] = kmeans.labels_

# merge london_grouped with london_data to add latitude/longitude for each neighborhood
london_grouped_clustering = london_grouped_clustering.join(venues_sorted.set_index('Street'), on='Street')

london_grouped_clustering.head(30) # check the last columns!

,Street,Avg_Price,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
196,ALBION SQUARE,2.450000e+06,-41.273758,173.289393,1,Café,Coffee Shop,Indian Restaurant,Pub,Restaurant,Bar,Park
390,ANHALT ROAD,2.435000e+06,51.480316,-0.166801,3,Pub,French Restaurant,Grocery Store,English Restaurant,Diner,Cocktail Bar,Plaza
405,ANSDELL TERRACE,2.250000e+06,51.499890,-0.189103,0,Juice Bar,Hotel,Pub,Restaurant,Indian Restaurant,Italian Restaurant,Clothing Store
422,APPLEGARTH ROAD,2.400000e+06,53.748654,-0.326670,3,Bar,Pub,Casino,Nightclub,Sandwich Place,Zoo Exhibit,Farmers Market
855,BARONSMEAD ROAD,2.375000e+06,51.477315,-0.239457,2,Food & Drink Shop,Nature Preserve,Restaurant,Farmers Market,Coffee Shop,Park,Pizza Place
981,BEAUCLERC ROAD,2.480000e+06,30.211452,-81.617981,1,Pizza Place,Spa,Automotive Shop,Doctor's Office,Fast Food Restaurant,Ethiopian Restaurant,Event Space
1102,BELVEDERE DRIVE,2.340000e+06,44.707562,-63.545599,2,Campground,Zoo Exhibit,Fast Food Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Factory
1215,BICKENHALL STREET,2.208500e+06,51.521201,-0.158908,0,Coffee Shop,Italian Restaurant,Gastropub,Restaurant,Hotel,Pizza Place,Café
1253,BIRCHLANDS AVENUE,2.217000e+06,51.448394,-0.160468,0,Pub,Lake,Coffee Shop,Chinese Restaurant,French Restaurant,Bakery,Train Station
1553,BRAMPTON GROVE,2.456875e+06,51.589961,-0.318525,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
# Create Map

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(london_grouped_clustering['Latitude'], london_grouped_clustering['Longitude'], london_grouped_clustering['Street'], london_grouped_clustering['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [46]:
london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 0\
                              , london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
405,2250000.0,Juice Bar,Hotel,Pub,Restaurant,Indian Restaurant,Italian Restaurant,Clothing Store
1215,2208500.0,Coffee Shop,Italian Restaurant,Gastropub,Restaurant,Hotel,Pizza Place,Café
1253,2217000.0,Pub,Lake,Coffee Shop,Chinese Restaurant,French Restaurant,Bakery,Train Station
2225,2200000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2638,2250000.0,Grocery Store,Pizza Place,Bakery,Pharmacy,Outdoor Sculpture,Supermarket,Coffee Shop


In [47]:
london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] ==1\
                              , london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
196,2450000.0,Café,Coffee Shop,Indian Restaurant,Pub,Restaurant,Bar,Park
981,2480000.0,Pizza Place,Spa,Automotive Shop,Doctor's Office,Fast Food Restaurant,Ethiopian Restaurant,Event Space
1553,2456875.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1980,2492500.0,Supermarket,English Restaurant,Rental Car Location,Gym,Coffee Shop,Dry Cleaner,Park
2136,2461000.0,Trail,Food,Zoo Exhibit,Fast Food Restaurant,Ethiopian Restaurant,Event Space,Exhibit


In [48]:
london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] ==2\
                              , london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
855,2375000.0,Food & Drink Shop,Nature Preserve,Restaurant,Farmers Market,Coffee Shop,Park,Pizza Place
1102,2340000.0,Campground,Zoo Exhibit,Fast Food Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Factory
2068,2375000.0,Pub,Park,Yoga Studio,Bakery,Hotel,Grocery Store,Ice Cream Shop
2129,2379652.7,Pub,Bakery,Hotel,Indian Restaurant,Ice Cream Shop,Hostel,Grocery Store
2944,2367500.0,Hotel,Pub,Garden,Café,Coffee Shop,Italian Restaurant,Chinese Restaurant


In [49]:
london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 3\
                              , london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
390,2435000.0,Pub,French Restaurant,Grocery Store,English Restaurant,Diner,Cocktail Bar,Plaza
422,2400000.0,Bar,Pub,Casino,Nightclub,Sandwich Place,Zoo Exhibit,Farmers Market
1632,2397132.0,Convenience Store,Italian Restaurant,Breakfast Spot,Grocery Store,Coffee Shop,Gym / Fitness Center,Cricket Ground
1797,2400000.0,Home Service,Zoo Exhibit,Fast Food Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit
1914,2445000.0,Construction & Landscaping,Dance Studio,Grocery Store,Bar,Flower Shop,Flea Market,Food & Drink Shop


In [50]:
london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] ==4 \
                              , london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
2242,2.300000e+06,Farm,Zoo Exhibit,Filipino Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Factory
2406,2.286679e+06,Italian Restaurant,Café,Pub,Coffee Shop,Bar,French Restaurant,Park
2686,2.287500e+06,Pub,Brewery,Art Museum,Gift Shop,Gym / Fitness Center,Flower Shop,Farm
3377,2.298000e+06,Hotel,Zoo Exhibit,Fast Food Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit
4285,2.265000e+06,Gym,Dry Cleaner,Trail,American Restaurant,Farmers Market,English Restaurant,Ethiopian Restaurant



# Results and Discussion section¶

First of all, even though the London Housing Market may be in a rut, it is still an "ever-green" for business affairs.

We may discuss our results under two main perspectives.

First, we may examine them according to neighborhoods/London areas. It is interesting to note that, although West London (Notting Hill, Kensington, Chelsea, Marylebone) and North-West London (Hampsted) might be considered highly profitable venues to purchase a real estate according to amenities and essential facilities surrounding such venues i.e. elementary schools, high schools, hospitals & grocery stores, South-West London (Wandsworth, Balham) and North-West London (Isliington) are arising as next future elite venues with a wide range of amenities and facilities. Accordingly, one might target under-priced real estates in these areas of London in order to make a business affair.

Second, we may analyze our results according to the five clusters we have produced. Even though, all clusters could praise an optimal range of facilities and amenities, we have found two main patterns. The first pattern we are referring to, i.e. Clusters 0, 2 and 4, may target home buyers prone to live in 'green' areas with parks, waterfronts. Instead, the second pattern we are referring to, i.e. Clusters 1 and 3, may target individuals who love pubs, theatres and soccer.



# Conclusion

To sum up, according to Bloomberg News, the London Housing Market is in a rut. It is now facing a number of different headwinds, including the prospect of higher taxes and a warning from the Bank of England that U.K. home values could fall as much as 30 percent in the event of a disorderly exit from the European Union. In this scenario, it is urgent to adopt machine learning tools in order to assist homebuyers clientele in London to make wise and effective decisions. As a result, the business problem we were posing was: how could we provide support to homebuyers clientele in to purchase a suitable real estate in London in this uncertain economic and financial scenario?



To solve this business problem, we clustered London neighborhoods in order to recommend venues and the current average price of real estate where homebuyers can make a real estate investment. We recommended profitable venues according to amenities and essential facilities surrounding such venues i.e. elementary schools, high schools, hospitals & grocery stores.



First, we gathered data on London properties and the relative price paid data were extracted from the HM Land Registry (http://landregistry.data.gov.uk/). Moreover, to explore and target recommended locations across different venues according to the presence of amenities and essential facilities, we accessed data through FourSquare API interface and arranged them as a data frame for visualization. By merging data on London properties and the relative price paid data from the HM Land Registry and data on amenities and essential facilities surrounding such properties from FourSquare API interface, we were able to recommend profitable real estate investments.



Second, The Methodology section comprised four stages: 1. Collect Inspection Data; 2. Explore and Understand Data; 3. Data preparation and preprocessing; 4. Modeling. In particular, in the modeling section, we used the k-means clustering technique as it is fast and efficient in terms of computational cost, is highly flexible to account for mutations in real estate market in London and is accurate.



Finally, we drew the conclusion that even though the London Housing Market may be in a rut, it is still an "ever-green" for business affairs. We discussed our results under two main perspectives. First, we examined them according to neighborhoods/London areas. although West London (Notting Hill, Kensington, Chelsea, Marylebone) and North-West London (Hampsted) might be considered highly profitable venues to purchase a real estate according to amenities and essential facilities surrounding such venues i.e. elementary schools, high schools, hospitals & grocery stores, South-West London (Wandsworth, Balham) and North-West London (Isliington) are arising as next future elite venues with a wide range of amenities and facilities. Accordingly, one might target under-priced real estates in these areas of London in order to make a business affair. Second, we analyzed our results according to the five clusters we produced. While Clusters 0, 2 and 4 may target home buyers prone to live in 'green' areas with parks, waterfronts, Clusters 1 and 3 may target individuals who love pubs, theatres and soccer.
